<a href="https://colab.research.google.com/github/dfieman/CRNC/blob/main/ErosionRateFinder_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#==================================================================================================================
# Code for getting the erosion rate of each cell. Uses linear and nonlinear diffusion and slope-dependent erosion #
# Main output is the erosion rate [cm/yr] and the concentration [at/g] of every cell. User can either input a
# catchment-averged erosion rate and/or measured Be-10, Al-26, or C-14 concentration.
#==================================================================================================================
##

import math
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
from scipy import interpolate, LowLevelCallable
import scipy.integrate as integrate
import multiprocessing
import sys
import datetime
from functools import partial
import os
import glob
import ctypes
import numba
from numba import jit
#import more_itertools as mit
from scipy.interpolate import Rbf
from scipy.interpolate import griddata
import random
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
from scipy.optimize import newton
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#############
# FUNCTIONS #
#############

# PRODUCTION RATE FUNCTIONS #
## Only calculates the production at the surface scaled to elevation and latitude.
## Spallation scaled to scheme by Lal/Stone
## Muons are simplified to one exponential function and scaled to scheme by Balco 2017

#Convert elevation to units of hPa (barometric pressure) and g/cm^2 (atmospheric pressure).
def h_units(h):
    #Constants for pressure
    Psl = 1013.25 #hPa; atmopheric pressure at sea level
    tempsl = 288.15 #K; temperature at sea level
    dTdz = 0.0065 #K/m; adiabatic lapse rate
    g = 9.80665 #m/s^2
    air = 0.0289644 #kg/mol; molecular mass of air
    gas = 8.31446 #J/mol/K; gas constant
    baro = Psl * np.exp(-1*(g*air/(gas*dTdz))*(np.log(tempsl)-np.log(tempsl-(dTdz*h))))
    atPres = 1.019716*(1013.25-baro)
    return baro, atPres

#Spallation scaling production equations from Stone 2000:
#Input argument l =latitude [deg], h = elevation [hPa], and Pref = reference production rate at SLHL [atoms/g/yr]
def spalProd(l,pressure_array):
    #Interpolation for latitute coefficient. See Table 1 Stone 2000
    coeff = np.array([(31.8518, 250.3193, -0.083393, 7.4260E-5, -2.2397E-8),(34.3699, 258.4759, -0.089807, 7.9457E-5, -2.3697E-8),(40.3153, 308.9894, -0.106248, 9.4508E-5, -2.8234E-8),(42.0983, 512.6857, -0.120551, 1.1752E-4, -3.8809E-8), (56.7733, 649.1343, -0.160859, 1.5463E-4, -5.033E-8),(69.0720, 832.4566, -0.199252, 1.9391E-4, -6.3653E-8),(71.8733, 863.1927, -0.207069, 2.0127E-4, -6.6043E-8)])
    a_coeff = coeff[:,0]
    b_coeff = coeff[:,1]
    c_coeff = coeff[:,2]
    d_coeff = coeff[:,3]
    e_coeff = coeff[:,4]
    lat = np.array([0,10,20,30,40,50,60])
    a_func = interpolate.interp1d(lat,a_coeff)
    b_func = interpolate.interp1d(lat,b_coeff)
    c_func = interpolate.interp1d(lat,c_coeff)
    d_func = interpolate.interp1d(lat,d_coeff)
    e_func = interpolate.interp1d(lat,e_coeff)
    Sp = a_func(l)+(b_func(l)*np.exp(-1*pressure_array/150))+(c_func(l)*pressure_array)+(d_func(l)*(pressure_array**2))+(e_func(l)*(pressure_array**3)) #Scaling factor
    Pref = [3.84,25.92,11.7]
    return 0.978*Sp*Pref[CRN]

def LeffCalculator(pressure_array,erosion_array):

    valid_values = erosion_array>0.0
    rows,cols = np.where(valid_values)

    baro = np.where(np.isnan(erosion_array),np.nan,pressure_array)
    baro_flat = baro.flatten()[~np.isnan(baro.flatten())]
    erosion_flat = erosion_array.flatten()[~np.isnan(erosion_array.flatten())]
    #print(len(baro_flat),len(erosion_flat))
    Leffmu = griddata((P_grid.flatten(),E_grid.flatten()),Leff[CRN].flatten(),(baro_flat,erosion_flat),method='nearest')

    Leff2D = np.full_like(baro,np.nan,dtype='float')
    for i,j,val in zip(rows,cols,Leffmu):
        Leff2D[i][j] = val

    return Leff2D

def muProd(pressure_array):
    return Pref_mu[CRN]*np.exp((1013.25-pressure_array)/mu_efold[CRN])

#Concentration if t is inf and at the surface.
def surfaceConcentration(pressure_array,erosion_array,spProd,muProd):
    Leffmu = LeffCalculator(pressure_array,erosion_array)
    #print('Leff mean:',np.nanmean(Leffmu))
    Csp = spProd/(decayConst[CRN]+(rho*erosion_array/sp_efold))
    Cmu = muProd/(decayConst[CRN]+(rho*erosion_array/Leffmu))
    return Csp+Cmu

# BEST-FIT FUNCTIONS #

#Find the best-fit K and Sc for diffusion if concentration is the input
def diffK_C(params,fx,fxx,fy,fyy,pressure_array,spProd,muProd):
    K,Sc = params

    curvature = fxx+fyy
    slope = np.sqrt((fx**2)+(fy**2))

    linearFlux = -1*(K*(fxx+fyy))*100
    linearErosion = np.where((curvature >= -0.0005), np.nan, linearFlux)

    nonlinearFlux = -1*(K*(((fxx+fyy)/(1-((slope/Sc)**2)))+(2*(((fx**2)*fxx)+((fy**2)*fyy)+((fx**2)*fyy)+((fy**2)*fxx))/((Sc**2)*((1-((slope/Sc)))**2)))))*100
    nonlinearErosion = np.where((curvature >= -0.0005), np.nan, nonlinearFlux)

    Leffmu_lin = LeffCalculator(pressure_array,linearErosion)
    Leffmu_nonlin = LeffCalculator(pressure_array,nonlinearErosion)

    C_total_linE = (spProd/(decayConst[CRN]+(rho*linearErosion/sp_efold)))+(muProd/(decayConst[CRN]+(rho*linearErosion/Leffmu_lin)))
    C_total_nonlinE = (spProd/(decayConst[CRN]+(rho*nonlinearErosion/sp_efold)))+(muProd/(decayConst[CRN]+(rho*nonlinearErosion/Leffmu_nonlin)))

    return abs(np.nanmean(C_total_linE)-C_measured)+abs(np.nanmean(C_total_nonlinE)-C_measured)
    #return abs(np.nanmedian(C_total_linE)-C_measured)+abs(np.nanmedian(C_total_nonlinE)-C_measured)

#Find the best-fit K and Sc for diffusion if erosion is the input
def diffK_E(params,fx,fxx,fy,fyy,spProd,muProd,eRate):
    K,Sc = params

    curvature = fxx+fyy
    slope = np.sqrt((fx**2)+(fy**2))

    linearFlux = -1*(K*(fxx+fyy))*100
    linearErosion = np.where((curvature >= -0.0005), np.nan, linearFlux)

    nonlinearFlux = -1*(K*(((fxx+fyy)/(1-((slope/Sc)**2)))+(2*(((fx**2)*fxx)+((fy**2)*fyy)+((fx**2)*fyy)+((fy**2)*fxx))/((Sc**2)*((1-((slope/Sc)))**2)))))*100
    nonlinearErosion = np.where((curvature >= -0.0005), np.nan, nonlinearFlux)

    return abs(np.nanmean(linearErosion)-eRate)+abs(np.nanmean(nonlinearErosion)-eRate)


#Find the best-fit K and Sc for slope-dependent erosion Mont&Brandon
def slopeK_C(params,slope,pressure_array,spProd,muProd):
    K,Sc = params

    slopeDependent = (K*(slope/(1-((slope/Sc)**2))))*100 #convert from m/yr to cm/yr
    slopeErosion = np.where((np.isinf(slopeDependent) | (slopeDependent <= 0.0)), np.nan, slopeDependent)

    Leffmu = LeffCalculator(pressure_array,slopeErosion)

    C_total = (spProd/(decayConst[CRN]+(rho*slopeErosion/sp_efold)))+(muProd/(decayConst[CRN]+(rho*slopeErosion/Leffmu)))

    return abs(np.nanmean(C_total)-C_measured)
    #return abs(np.nanmedian(C_total)-C_measured)

def slopeK_E(params,slope,spProd,muProd,eRate):
    K,Sc = params

    slopeDependent = (K*(slope/(1-((slope/Sc)**2))))*100 #convert from m/yr to m/yr
    slopeErosion = np.where((np.isinf(slopeDependent) | (slopeDependent <= 0.0)), np.nan, slopeDependent)

    return abs(np.nanmean(slopeErosion)-eRate)


# Solve for uniform E with the measured Concentration
def uniformEFinder(erosion,pressure_array,spProd,muProd):
    uniformErosion = np.where(~np.isnan(pressure_array),erosion,pressure_array)

    Leffmu = LeffCalculator(pressure_array,uniformErosion)

    C_total = (spProd/(decayConst[CRN]+(rho*uniformErosion/sp_efold)))+(muProd/(decayConst[CRN]+(rho*uniformErosion/Leffmu)))

    return abs(np.nanmean(C_total)-C_measured)

# EROSION-RATE FUNCTIONS #

# Create erosion rates based on best fit K and Sc values
@jit(nopython = True)
def erosionRate(fx,fy,fxx,fyy,diffK,slopeK,diffSc,slopeSc,uniform_erosion):
    slope = np.sqrt((fx**2)+(fy**2))
    curvature = fxx+fyy

    #limit curvature so greater than -0.0005 is flat

    linearFlux = -1*(diffK*(fxx+fyy))*100
    linear = np.where((curvature >= -0.0005), np.nan, linearFlux)

    nonlinearFlux = -1*(diffK*(((fxx+fyy)/(1-((slope/diffSc)**2)))+(2*(((fx**2)*fxx)+((fy**2)*fyy)+((fx**2)*fyy)+((fy**2)*fxx))/((diffSc**2)*((1-((slope/diffSc)))**2)))))*100
    nonlinear = np.where((curvature >= -0.0005), np.nan, nonlinearFlux)


    slopeDependent = slopeK*(slope/(1-((slope/slopeSc)**2)))*100 #Montgomery&Brandon2002
    slopeErosion = np.where((np.isinf(slopeDependent) | (slopeDependent <= 0.0)), np.nan, slopeDependent)

    uniform = np.where(~np.isnan(slope),uniform_erosion,slope)
    #return linearErosion, nonlinearErosion, slopeEroion
    return linear,nonlinear,slopeErosion,uniform

# Uncertainties using guasssian propagation

def erosionUncertainty(pressure_array,erosion_array):
    pressure = np.where(np.isnan(erosion_array),np.nan,pressure_array)
    Sp = spalProd(latitude,pressure)/Pref[CRN]
    dEdP = sp_efold*Sp/(rho*C_measured)
    dEdC = -sp_efold*Sp*Pref[CRN]/(rho*C_measured**2)
    return np.sqrt(((dEdP*Pref_unc[CRN])**2)+((dEdC*C_measured_unc)**2))

def concUncertainty(pressure_array,erosion_array,erosion_array_unc):
    pressure = np.where(np.isnan(erosion_array),np.nan,pressure_array)
    Sp = spalProd(latitude,pressure)/Pref[CRN]
    dCdP = Sp/(decayConst[CRN]+(rho*erosion_array/sp_efold))
    dCdE = -rho*Sp*Pref[CRN]/(((decayConst[CRN]+(rho*erosion_array/sp_efold))**2)*sp_efold)
    return np.sqrt(((dCdP*Pref_unc[CRN])**2)+((dCdE*erosion_array_unc)**2))

# FUNCTIONS FOR INITAITING INPUTS AND CREATING TIFS #

#Open TIFs
def gdal_open(DEM_name,folder):
    raster = gdal.Open(folder+DEM_name)
    nodata = raster.GetRasterBand(1).GetNoDataValue()
    array = raster.GetRasterBand(1).ReadAsArray()
    array[array==nodata]=np.nan
    return array

#Make numpy arrays into rasters
def RasterMaker(array,output_file,reference_tiff):
    reference = gdal.Open(reference_tiff)
    nodata = reference.GetRasterBand(1).GetNoDataValue()
    array[np.isnan(array)]=nodata
    driver = gdal.GetDriverByName('GTiff')
    raster = driver.CreateCopy(output_file, reference, strict=0, options=["TILED=YES","COMPRESS=PACKBITS"])
    raster.GetRasterBand(1).WriteArray(array)
    raster.GetRasterBand(1).SetNoDataValue(nodata)
    raster = None



In [ ]:
######################################
# Paramters for production functions # Dont touch this
######################################
CRNs = ['Be10','Al26','C14']
halfLife = [1.36e6, 7.17e5, 5700] #Half life
decayConst = np.log(2)/halfLife #Decay constant
sp_efold = 160 #effective attenuation length into rock [g/cm2]

#Pre-calculated Leff grid from Balco 2016 for muon efolding length
#Erate in cm/yr
Erate = [0,0.0001,0.000158489,0.000251189,0.000398107,0.000630957,0.001,0.001584893,0.002511886,0.003981072,0.006309573,0.01,0.015848932,0.025118864,0.039810717,0.063095734,0.1,0.158489319,0.251188643,0.398107171,0.630957344,1]
Pressure = [450,480,510,540,570,600,630,660,690,720,750,780,810,840,870,900,930,960,990,1020]

P_grid,E_grid = np.meshgrid(Pressure,Erate)
#Rows are erosion rate, columns are pressure
Leff26 = np.genfromtxt('/content/drive/MyDrive/Topo_Analysis/Leff26.csv',delimiter=',')
Leff10 = np.genfromtxt('/content/drive/MyDrive/Topo_Analysis/Leff10.csv',delimiter=',')
Leff = [Leff10,Leff26]

#For indexing
Be10 = 0
Al26 = 1
C14 = 2

#Reference production rate in atoms/g/yr for spallation production and muon production.
#For spallation 10Be we use number from Putnam et al, C14 from Schaefer et al 2014. We use a 6.75 ratio for 26/10.
# Muon is found from Balco 2017. No uncertainties are given
Pref = [3.84,25.92, 11.7]
Pref_unc = [0.08,0.54,0.9]
Pref_mu = [0.0735,0.6764,3.067] #atoms/g/yr
mu_efold = [299.2,288.0,267.8] #hPa

###############
# USER INPUTS #
###############

## Which CRN would you like to model?
CRN = Be10 #Option are 'Be10','Al26', or 'C14'. Can only do one at a time for now. Dont put quotes around it

rho = 2.65 #rock density [g/cm2]
latitude = 42 #[deg]

#DEM_name = 'Hapuku'
#DEM_name = 'Kowhai'
DEM_name = 'Stanley'
#DEM_name = 'Baton'
print(DEM_name)
# #Where rasters are saved
folder = '/content/drive/MyDrive/Topo_Analysis/'
erosion_folder = '/content/drive/MyDrive/Erosion_Rasters/'
rasterformat = '.tif'

#C_measured = 3127 if DEM_name == 'Hapuku' else 2375 # Measured concentration
#C_measured_unc = 226 if DEM_name == 'Hapuku' else 190
C_measured = 29400 if DEM_name == 'Baton' else 60000 # Measured concentration
C_measured_unc = 3300 if DEM_name == 'Baton' else 2800

elevation_raster = f'{DEM_name}_Masked_Wshd'
slope_raster = f'{DEM_name}_r6_Slope'
curvature_raster = f'{DEM_name}_r6_Curvature'
derivative_rasters = [f'{DEM_name}_r6_dx',f'{DEM_name}_r6_dxx',f'{DEM_name}_r6_dy',f'{DEM_name}_r6_dyy']
hilltop_raster = f'{DEM_name}_r6_HTCurvature'

Stanley


# New Section

In [ ]:
###########################################
# Open rasters and convert  to numpy arrays
###########################################

elevationArray = gdal_open(elevation_raster+rasterformat,folder)
slopeArray = gdal_open(slope_raster+rasterformat,folder)
curvatureArray = gdal_open(curvature_raster+rasterformat,folder)
derivativeArrays = [gdal_open(raster+rasterformat,folder) for raster in derivative_rasters]
hilltopArray = gdal_open(hilltop_raster+rasterformat,folder)
"""
Baton_erosion_names = ['Baton_K0.000_Sc7.00_SlopeErosion','Baton_K0.019_LinErosion','Baton_K0.019_Sc6.99_NonlinErosion']
Baton_concentration_names = ['Baton_Be10_uniformE0.02','Baton_Be10_slopeE0.02','Baton_Be10_linE0.07','Baton_Be10_nonlinE12.39']
"""
dx = derivativeArrays[0]
dxx = derivativeArrays[1]
dy = derivativeArrays[2]
dyy = derivativeArrays[3]

print("opened rasters")

opened rasters


In [ ]:
################
# CALCULATIONS #
################

#Inital guesses. These are super important. If they're not within about 10% of the value good luck
#Sc_x0 = 2.0 #Change to just below max slope #7.0
#uniform_x0 = 0.005 #0.015
#diffK_x0 = 0.005 #0.0005
#slopeK_x0 = 0.0001 #0.0003
Sc_x0 = 2.0
uniform_x0 = 0.006
diffK_x0 = 0.008
slopeK_x0 = 0.0002

# Convert elevation [m] (2D array) barometric pressure [hPa] and atmospheric pressure [g/cm2]
pressure_array, atPres = h_units(elevationArray)

# Solve for spallation production at the surface. Inputs l = latitude (scalar), h = baro (2D array), Pref = reference production rate.
Psp = spalProd(latitude,pressure_array)
print('solved spallation production')

# Solve for muon production at surface:
Pmu = muProd(pressure_array)
print('solved muon production')

solved spallation production
solved muon production


In [ ]:
"""
Bat_uniform_erosion = 0.01806

Baton_erosion_arrays = [gdal_open(name+rasterformat,erosion_folder) for name in Baton_erosion_names]
Baton_erosion_arrays.insert(0,np.full_like(Baton_erosion_arrays[0],Bat_uniform_erosion))
Baton_conc_arrays = [gdal_open(name+rasterformat,erosion_folder) for name in Baton_concentration_names]

E_type = ['Uniform','Slope','Linear','Nonlinear']

E_mean = [np.nanmean(erosion) for erosion in Baton_erosion_arrays]
E_median = [np.nanmedian(erosion) for erosion in Baton_erosion_arrays]
C_mean = [np.nanmean(concentration) for concentration in Baton_conc_arrays]
C_median = [np.nanmedian(concentration) for concentration in Baton_conc_arrays]


erosionArrays_unc = [erosionUncertainty(pressure_array,erosion) for erosion in Baton_erosion_arrays]
concArrays_unc = [concUncertainty(pressure_array,erosion,erosion_unc) for erosion,erosion_unc in zip(Baton_erosion_arrays,erosionArrays_unc)]
print(len(erosionArrays_unc))
print(len(concArrays_unc))

E_mean_unc = [np.sqrt(np.nansum(erosion_unc**2)/np.count_nonzero(~np.isnan(erosion_unc))) for erosion_unc in erosionArrays_unc]
E_median_unc = [np.nanmedian(erosion_unc) for erosion_unc in erosionArrays_unc]

C_mean_unc = [np.sqrt(np.nansum(conc_unc**2)/np.count_nonzero(~np.isnan(conc_unc))) for conc_unc in concArrays_unc]
C_median_unc = [np.nanmedian(conc_unc) for conc_unc in concArrays_unc]

mean_HT = np.nanmean(hilltopArray)
K_HT_uniform = (-(1/100)*Bat_uniform_erosion/mean_HT)
#E_list.append(uniformE)
K_HT_linear = ((-1/100)*E_mean[2]/mean_HT)
#E_list.append(E_list[0])

print('E_type:',E_type)
print('Mean E:',E_mean)
print('Mean E unc:',E_mean_unc)
print('Median E:',E_median)
print('Median E unc:',E_median_unc)
print('Mean C:',C_mean)
print('Mean C unc:',C_mean_unc)
print('Median C:',C_median)
print('Median C unc:',C_median_unc)
print('K-uni:',K_HT_uniform)
print('K-lin:',K_HT_linear)
"""

"\nBat_uniform_erosion = 0.01806\n\nBaton_erosion_arrays = [gdal_open(name+rasterformat,erosion_folder) for name in Baton_erosion_names]\nBaton_erosion_arrays.insert(0,np.full_like(Baton_erosion_arrays[0],Bat_uniform_erosion))\nBaton_conc_arrays = [gdal_open(name+rasterformat,erosion_folder) for name in Baton_concentration_names]\n\nE_type = ['Uniform','Slope','Linear','Nonlinear']\n\nE_mean = [np.nanmean(erosion) for erosion in Baton_erosion_arrays]\nE_median = [np.nanmedian(erosion) for erosion in Baton_erosion_arrays]\nC_mean = [np.nanmean(concentration) for concentration in Baton_conc_arrays]\nC_median = [np.nanmedian(concentration) for concentration in Baton_conc_arrays]\n\n\nerosionArrays_unc = [erosionUncertainty(pressure_array,erosion) for erosion in Baton_erosion_arrays]\nconcArrays_unc = [concUncertainty(pressure_array,erosion,erosion_unc) for erosion,erosion_unc in zip(Baton_erosion_arrays,erosionArrays_unc)]\nprint(len(erosionArrays_unc))\nprint(len(concArrays_unc))\n\nE_me

In [ ]:
print(np.nanmean(Psp))

5.0991683


In [ ]:
"""
diffSc = 2.006188672
diffK = 0.008

slopeK = 0.0001
slopeSc = 2.0
uniformE = 0.006329513225

all_erosionArrays = erosionRate(dx,dy,dxx,dyy,diffK,slopeK,diffSc,slopeSc,uniformE)
erosionArrays = [all_erosionArrays[0],all_erosionArrays[1]]
concArrays = []
for erosion in erosionArrays:
  print('mean E:',np.nanmean(erosion))
  print('median E:',np.nanmedian(erosion))
  conc = surfaceConcentration(pressure_array,erosion,Psp,Pmu)
  concArrays.append(conc)
print(len(concArrays))
for conc in concArrays:
  print('mean C:',np.nanmean(conc))
  print('median C:',np.nanmedian(conc))
"""


mean E: 0.01666152326077555
median E: 0.01133096106350422
mean E: 0.14367979397075098
median E: 0.014316187308054295
2
mean C: 69784.61897203185
median C: 34097.233398736455
mean C: 55977.010790944645
median C: 27244.000674658193


In [ ]:
linear_file = DEM_name +"_K"+f"{diffK:.3f}"+"_LinErosion"
nonlinear_file = DEM_name +"_K"+f"{diffK:.3f}"+"_Sc"+f"{diffSc:.2f}"+"_NonlinErosion"
erosion_files = [linear_file,nonlinear_file]
for each_file, each_array in zip(erosion_files,erosionArrays):
    print("Saving as",each_file)
    RasterMaker(each_array,erosion_folder+each_file+rasterformat,folder+elevation_raster+rasterformat)
print("Made erosion arrays into tifs")
C_lin_file = DEM_name + f'_{CRNs[CRN]}'+f'_linE0.02'
C_nonlin_file = DEM_name + f'_{CRNs[CRN]}'+f'_nonlinE0.14'
C_files = [C_lin_file,C_nonlin_file]
for each_file, each_array in zip(C_files,concArrays):
    print("Saving as",each_file)
    RasterMaker(each_array,erosion_folder+each_file+rasterformat,folder+elevation_raster+rasterformat)
print("Made concentration arrays into tifs")

Saving as Stanley_K0.008_LinErosion
Saving as Stanley_K0.008_Sc2.01_NonlinErosion
Made erosion arrays into tifs
Saving as Stanley_Be10_linE0.02
Saving as Stanley_Be10_nonlinE0.14
Made concentration arrays into tifs


In [ ]:
# Calculating best-fit K and Sc. Outputs in meters
K_list = []
Sc_list = []
E_list = []
C_list = []
E_median_list = []
C_median_list = []
Input = ['Be-10','Be-10','Be-10','Be-10','Erosion','Erosion','Erosion','Erosion','HT_Curv','HT_Curv'] # Change this if not inputting erosion
E_type = ['Linear','Nonlinear','Slope','Uniform','Linear','Nonlinear','Slope','Uniform','Diffusion','Diffusion']

all_erosions = []

# # Best-fit diffusion K and Sc for concentration as input. Tolerance is 1000 atoms/g
args = (dx,dxx,dy,dyy,pressure_array,Psp,Pmu)
data_x0 = [diffK_x0,Sc_x0]
diffK,diffSc = newton(diffK_C,data_x0,args=args,tol = 1000)
K_list.extend([diffK]*2)
Sc_list.extend([np.nan,diffSc])
print('solved K and Sc for diffusion')
print('K and Sc:',diffK,diffSc)
# # Best-fit slope-dependent K and Sc for concentration as input.
args = (slopeArray,pressure_array,Psp,Pmu)
data_x0 = [slopeK_x0,Sc_x0]
slopeK,slopeSc = newton(slopeK_C,data_x0,args=args)
K_list.append(slopeK)
Sc_list.append(slopeSc)
print('solved K and Sc for slope-dependent')
print('slope K and Sc:',slopeK,slopeSc)
# # Best-fit uniform erosion rate for concentration as input
args = (pressure_array,Psp,Pmu)
data_x0 = uniform_x0
uniformE = newton(uniformEFinder,data_x0,args=args,maxiter = 1000) #output is cm/yr
K_list.append(np.nan)
Sc_list.append(np.nan)
print('solved uniform E:',uniformE)

# Create the erosion and surface concentration arrays based on the best-fit parameters
erosionArrays = erosionRate(dx,dy,dxx,dyy,diffK,slopeK,diffSc,slopeSc,uniformE)
for erosion in erosionArrays:
    E_list.append(np.nanmean(erosion))
    E_median_list.append(np.nanmedian(erosion))
    all_erosions.append(erosion)
concArrays = [surfaceConcentration(pressure_array,erosion,Psp,Pmu) for erosion in erosionArrays]
for conc in concArrays:
    C_list.append(np.nanmean(conc))
    C_median_list.append(np.nanmedian(conc))

# # Save erosion and concentration arrays as tiffs
linear_file = DEM_name +"_K"+f"{diffK:.3f}"+"_LinErosion"
nonlinear_file = DEM_name +"_K"+f"{diffK:.3f}"+"_Sc"+f"{diffSc:.2f}"+"_NonlinErosion"
slope_file = DEM_name +"_K"+f"{slopeK:.3f}"+"_Sc"+f"{slopeSc:.2f}"+"_SlopeErosion"
erosion_files = [linear_file,nonlinear_file,slope_file]
for each_file, each_array in zip(erosion_files,erosionArrays):
    print("Saving as",each_file)
    RasterMaker(each_array,erosion_folder+each_file+rasterformat,folder+elevation_raster+rasterformat)
print("Made erosion arrays into tifs")
C_lin_file = DEM_name + f'_{CRNs[CRN]}'+f'_linE{E_list[-4]:.2f}'
C_nonlin_file = DEM_name + f'_{CRNs[CRN]}'+f'_nonlinE{E_list[-3]:.2f}'
C_slope_file = DEM_name + f'_{CRNs[CRN]}'+f'_slopeE{E_list[-2]:.2f}'
C_uniform_file = DEM_name + f'_{CRNs[CRN]}'+f'_uniformE{E_list[-1]:.2f}'
C_files = [C_lin_file,C_nonlin_file,C_slope_file,C_uniform_file]
for each_file, each_array in zip(C_files,concArrays):
    print("Saving as",each_file)
    RasterMaker(each_array,erosion_folder+each_file+rasterformat,folder+elevation_raster+rasterformat)
print("Made concentration arrays into tifs")



solved K and Sc for diffusion
K and Sc: 0.01646214350422366 2.0251849509055257


KeyboardInterrupt: 

In [ ]:


## Best-fit diffusion K and Sc for erosion as input. Tolerance is 0.02 cm/yr. Change uniformE if necessary
args = (dx,dxx,dy,dyy,Psp,Pmu,uniformE)
data_x0 = [diffK_x0,Sc_x0] # initial guesses for K and Sc
diffK,diffSc = newton(diffK_E,data_x0,args=args,tol=0.02)
K_list.extend([diffK]*2)
Sc_list.extend([np.nan,diffSc])
print('solved best-fit diffusion parameters')
# # Best-fit slope-dependent K and Sc for erosion as input
args = (slopeArray,Psp,Pmu,uniformE)
data_x0 = [0.004,Sc_x0] #inital guess
slopeK,slopeSc = newton(slopeK_E,data_x0,args=args)
K_list.append(slopeK)
Sc_list.append(slopeSc)
print('solves best-fit slope parameters')
erosionArrays_2 = erosionRate(dx,dy,dxx,dyy,diffK,slopeK,diffSc,slopeSc,uniformE)
for erosion in erosionArrays_2:
    E_list.append(np.nanmean(erosion))
    E_median_list.append(np.nanmedian(erosion))
    all_erosions.append(erosion)
concArrays_2 = [surfaceConcentration(pressure_array,erosion,Psp,Pmu) for erosion in erosionArrays_2]
for conc in concArrays_2:
    C_list.append(np.nanmean(conc))
    C_median_list.append(np.nanmedian(conc))
K_list.append(np.nan)
Sc_list.append(np.nan)

print('K:',K_list)
print('Sc',Sc_list)
print('E',E_list)
print('C',C_list)

In [ ]:
print('K:',K_list)
print('Sc',Sc_list)
print('E',E_list)
print('C',C_list)

In [ ]:
print('C:',len(C_list))
print('E:',len(E_list))
print('C_median:',len(C_median_list))
print('E_median:',len(E_median_list))

##Solve for uncertainties
erosionArrays_unc = [erosionUncertainty(pressure_array,erosion) for erosion in all_erosions]
concArrays_unc = [concUncertainty(pressure_array,erosion,erosion_unc) for erosion,erosion_unc in zip(all_erosions,erosionArrays_unc)]
print(len(erosionArrays_unc))
print(len(concArrays_unc))

E_mean_unc = [np.sqrt(np.nansum(erosion_unc**2)/np.count_nonzero(~np.isnan(erosion_unc))) for erosion_unc in erosionArrays_unc]
E_median_unc = [np.nanmedian(erosion_unc) for erosion_unc in erosionArrays_unc]

C_mean_unc = [np.sqrt(np.nansum(conc_unc**2)/np.count_nonzero(~np.isnan(conc_unc))) for conc_unc in concArrays_unc]
C_median_unc = [np.nanmedian(conc_unc) for conc_unc in concArrays_unc]

print('C_mean_unc:',len(C_mean_unc))
print('C median unc:',len(C_median_unc))
print('E mean unc:',len(E_mean_unc))
print('E median unc',len(E_median_unc))

In [ ]:
#print([np.nanmean(conc)for conc in concArrays_unc])
#print([np.nanmean(e) for e in erosionArrays_unc])
#print([np.sqrt(np.nansum(conc_unc**2)/np.count_nonzero(~np.isnan(conc_unc))) for conc_unc in concArrays_unc])
#print([np.sqrt(np.nansum(erosion_unc**2)/np.count_nonzero(~np.isnan(erosion_unc))) for erosion_unc in erosionArrays_unc])

In [ ]:
Sc_list.extend([np.nan]*2) # so curvature erosion does not have a Sc value
C_list.extend([np.nan]*2)
C_mean_unc.extend([np.nan]*2)
C_median_list.extend([np.nan]*2)
C_median_unc.extend([np.nan]*2)
E_median_list.extend([np.nan]*2)
E_median_unc.extend([np.nan]*2)
E_mean_unc.extend([np.nan]*2)
mean_HT = np.nanmean(hilltopArray)
K_list.append(-(1/100)*uniformE/mean_HT)
E_list.append(uniformE)
K_list.append((-1/100)*E_list[0]/mean_HT)
E_list.append(E_list[0])

In [ ]:
CSV_data = {
    'Input':Input,
    'Erosion_type': E_type,
    'K_[m2/yr]': K_list,
    'Sc': Sc_list,
    'Mean_E_[cm/yr]':E_list,
    'Mean_E_unc':E_mean_unc,
    'Median_E_[cm/yr]':E_median_list,
    'Median_E_unc':E_median_unc,
    'Mean_C_[at/g]':C_list,
    'Mean_C_unc':C_mean_unc,
    'Median_C_[at/g]':C_median_list,
    'Median_C_unc':C_median_unc
    }
#print(CSV_data)

In [ ]:
#print(C_mean_unc)

In [ ]:
print('Input:',len(Input))
print('E_typ:',len(E_type))
print('K_list:',len(K_list))
print('Sc_lis:',len(Sc_list))
print('E_list:',len(E_list))
print('C_list:',len(C_list))
print('E_unc:',len(E_mean_unc))
print('E_median:',len(E_median_list))
print('E_median_unc:',len(E_median_unc))
print('C_mean_unc:',len(C_mean_unc))
print('C_median:',len(C_median_list))
print('C_median_unc:',len(C_median_unc))

In [ ]:
print(E_mean_unc)
print(E_median_list)
print(E_median_unc)
print(C_list)
print(C_mean_unc)
print(C_median_list)
print(C_median_unc)

In [ ]:
pd.DataFrame(CSV_data).to_csv(erosion_folder+DEM_name+'_E_C_results.csv',index=False)
print("made csv")